In [16]:
# 날짜 2개 (2021-01-29, 2021-01-30)에 대해 로그처럼 순차적인 이벤트를 생성하는 함수
import pandas as pd, random, os, sqlite3
from datetime import datetime, timedelta
machines = ["CNC", "MCT", "TCT", "Boring", "Grinding", "EDM", "Mill-Turn"]
operations = ["error", "stopped", "operating"]

def generate_log(start_date_str="2021-01-29", days=2, entries_per_day=100):
    start_date = datetime.strptime(start_date_str, "%Y-%m-%d")
    logs = []

    for day_offset in range(days):
        day_date = start_date + timedelta(days=day_offset)
        current_time = datetime.combine(day_date, datetime.min.time())  # 00:00 시작
        prev_operation = prev_machine = None
        count = 0

        while count < entries_per_day:
            # 시간 증가
            current_time += timedelta(minutes=random.randint(5, 20))

            # 날짜가 넘어가면 종료
            if current_time.date() != day_date.date():
                break

            # 중복 조합 방지
            attempts = 0
            while True:
                operation = random.choice(operations)
                machine = random.choice(machines)
                if not (operation == prev_operation and machine == prev_machine):
                    break
                attempts += 1
                if attempts > 10:
                    break

            logs.append([
                current_time.year, current_time.month, current_time.day,
                current_time.hour, current_time.minute,
                operation, machine
            ])
            prev_operation, prev_machine = operation, machine
            count += 1

    return pd.DataFrame(logs, columns=["Year", "Month", "Day", "Hour", "Minute", "operation", "Machine"])




def saver(df, db_dir_path="./project/data/factory"):
    # 3.1. 컬럼 이름 정리
    df.columns = df.columns.str.strip()
    # 4. DB 이름 생성
    db_name = generate_db_name_from_machine_data(df)
    db_path = os.path.join(db_dir_path, db_name)
    # 5. SQLite 저장
    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        Name = "logs"  # 테이블 이름 고정
        cursor.execute(f"DROP TABLE IF EXISTS {Name}")
        df.to_sql(Name, conn, index=False)
        row_count = cursor.execute(f"SELECT COUNT(*) FROM {Name}").fetchone()[0]
        conn.commit()
        conn.close()
    except Exception as e:
        return {"error": f"DB 저장 실패: {str(e)}"}

def generate_db_name_from_machine_data(df): 

    # 기계 이름 추출 (모든 행 동일하다고 가정)
    machinename = 'log'

    # 시작 및 종료 시간 추출
    start = df.iloc[0]
    end = df.iloc[-1]

    def to_datetime_str(row):
        return f"{row['Year']:04}{row['Month']:02}{row['Day']:02}"
    
    start_str = to_datetime_str(start)
    end_str = to_datetime_str(end)

    db_name = f"{machinename}_{start_str}_{end_str}.db"
    return db_name

In [17]:
def make_log_data(start_list):
    for start_date_str in start_list:
        df = generate_log(start_date_str=start_date_str, days=2, entries_per_day=100)
        saver(df)

if __name__ == "__main__":
    starts = [datetime(2021, 1, 31, 0, 0)+ timedelta(days=2*i) for i in range(10)]
    starts = [start.strftime("%Y-%m-%d") for start in starts]
    make_log_data(starts)
    print("Log data generation completed.")

Log data generation completed.
